```{qu{python}
import yaml
import pywt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
data_path = '../aggr_data.csv'
data = pd.read_csv(data_path)

# Display the first few rows of the dataframe to understand its structure
data.head()

# Since the focus is on demonstrating the process, we'll select one participant as an example.
example_participant_id = data['PART_ID'].unique()[0]
example_data = data[data['PART_ID'] == example_participant_id]['mean_corr'].values

# DWT: Discrete Wavelet Transform
wavelet = 'db4'  # Daubechies wavelet with 4 vanishing moments, chosen for its good balance between time and frequency localization.
levels = pywt.dwt_max_level(data_len=len(example_data), filter_len=pywt.Wavelet(wavelet).dec_len)

# Perform the decomposition
coeffs = pywt.wavedec(data=example_data, wavelet=wavelet, level=levels)

# Plot the coefficients
plt.figure(figsize=(10, 6))
for i, coeff in enumerate(coeffs):
    plt.subplot(len(coeffs), 1, i+1)
    plt.plot(coeff)
    plt.title(f'Level {i} DWT Coefficients')
plt.tight_layout()
plt.show()

```


In [ ]:
# Synthetic sample data: Replace with your actual data
np.random.seed(0)  # For reproducibility
time_series = np.random.randn(150, 100)  # 150 participants, 100 time points each

In [ ]:
# Example using Discrete Wavelet Transform on the first participant's data
participant = time_series[0, :]  # First participant's data

# Perform DWT
coeffs = pywt.wavedec(participant, 'db1', level=None)  # 'db1' denotes Daubechies wavelet with 1 vanishing moment

# Plot the coefficients at each level
plt.figure(figsize=(12, 8))
for i, coeff in enumerate(coeffs):
    plt.subplot(len(coeffs), 1, i+1)
    plt.plot(coeff)
    plt.title(f'DWT Level {i}')
plt.tight_layout()
plt.show()

In [ ]:
variances = [np.var(coeff) for coeff in coeffs]

plt.figure(figsize=(8, 4))
plt.plot(variances, marker='o')
plt.xlabel('Decomposition Level')
plt.ylabel('Variance of Wavelet Coefficients')
plt.title('Variance Across Decomposition Levels')
plt.xticks(range(len(variances)))
plt.grid(True)
plt.show()

In [ ]:
# Load the data
data_path = '../aggr_data.csv'
data = pd.read_csv(data_path)

# Initialize a dictionary to hold variance data
variances_across_participants = {}

# Define the wavelet to use
wavelet = 'db4'

# Loop over each participant
for part_id, group in data.groupby('PART_ID'):
    time_series = group['mean_corr'].values
    coeffs = pywt.wavedec(time_series, wavelet, mode='per', level = None)
    variances = [np.var(coeff) for coeff in coeffs]
    variances_across_participants[part_id] = variances

In [ ]:
pywt.dwt_max_level()

In [ ]:
# Convert the variances to a DataFrame for easier analysis
variance_df = pd.DataFrame(variances_across_participants).T

# Calculate the mean variance across participants for each decomposition level
mean_variances = variance_df.mean()

# Plot the mean variance across decomposition levels
plt.figure(figsize=(10, 6))
plt.plot(mean_variances, marker='o', linestyle='-')
plt.xlabel('Decomposition Level')
plt.ylabel('Mean Variance of Wavelet Coefficients')
plt.title('Mean Variance Across Participants at Each Decomposition Level')
plt.grid(True)
plt.show()

# This plot can help identify if there's a consistent level where variance (and thus potential periodicity) is more pronounced.

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp

# Assuming variance_df is your DataFrame of variances across participants with rows as participants and columns as decomposition levels

def permutation_test(data, n_permutations=1000):
    """
    Performs a permutation test on the data to assess the significance of the observed mean variance.
    
    Parameters:
    - data: DataFrame containing the variances of wavelet coefficients across participants
    - n_permutations: Number of permutations to perform
    
    Returns:
    - p_values: P-values for each decomposition level, assessing the significance of the observed mean variance
    """
    observed_means = data.mean()
    p_values = pd.Series(index=observed_means.index, dtype=float)
    
    for level in data.columns:
        observed_mean = observed_means[level]
        distribution = []
        
        for _ in range(n_permutations):
            # Shuffle the data within each level
            shuffled = data[level].sample(frac=1, replace=False).reset_index(drop=True)
            distribution.append(shuffled.mean())
        
        # Calculate the p-value as the proportion of shuffled means that are greater than or equal to the observed mean
        p_value = (np.sum(distribution >= observed_mean) + 1) / (n_permutations + 1)
        p_values[level] = p_value
    
    return p_values

# Perform the permutation test
p_values = permutation_test(variance_df, n_permutations=1000)

# Display the p-values
print(p_values)